In [111]:
# Initial imports
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pathlib import Path
# from dotenv import load_dotenv
# import alpaca_trade_api as tradeapi
# from MCForecastTools_fp import MCSimulation
import numpy as np
import datetime
from datetime import date
import logging
import csv
import json


%matplotlib inline

In [112]:
# #Load .env enviroment variables
# load_dotenv()

In [113]:
bpip_delta=datetime.timedelta(weeks=1)
bpip_pair_delta=datetime.timedelta(weeks=4)
bpip_delta10=datetime.timedelta(weeks=12)
bpip_delta30=datetime.timedelta(weeks=205)

# ISO 8601 week as number with Monday as the first day of the week then change date from string to int
bpip_date = date.today()
bpip_date_cast = bpip_date + bpip_delta10
bpip_date_hist = bpip_date - bpip_delta30
bpip_day_fdiff = (bpip_date_cast - bpip_date) / 7
bpip_day_hdiff = (bpip_date - bpip_date_hist) / 7
bpip_fdiff = bpip_day_fdiff.days
bpip_hdiff = bpip_day_hdiff.days


In [114]:
#Loop through url calendars for forcast data
count=0
df_ffxevents_list = []
while(count < bpip_fdiff):
    bpip_dfdelta=datetime.timedelta(weeks=count)
    bpip_date_dfcast = bpip_date + bpip_dfdelta
    print(f"Forecast Date: {bpip_date_dfcast}")
    bpip_wk_dfcaststr = bpip_date_dfcast.strftime("%V")
    print(f"Forecast Week String: {bpip_wk_dfcaststr}")
    bpip_yr_dfcaststr = bpip_date_dfcast.strftime("%Y")
    print(f"Current Year String: {bpip_yr_dfcaststr}")
    bpip_yr_dfcastiso = bpip_date_dfcast.isocalendar()[0]
    print(f"Current Year: {bpip_yr_dfcastiso}")
    # get webpage and make the soup
    babe_url = f"https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week={bpip_yr_dfcastiso}-W{bpip_wk_dfcaststr}"
    print(babe_url)
    baseURL = requests.get(babe_url)
    fxcal_soup = BeautifulSoup(baseURL.content, 'html.parser')

    #Get all tag decendents 
    fxcal = fxcal_soup.find(class_="Table-module__table___2Nidf")
    fxcal_tbl_days = fxcal.find_all(class_="Section-module__container___2mwAW")

    #Loop through html classes and div tag elements
    #df_ffxevents_list = []
    for x in fxcal_tbl_days[0:]:
        pd_news_day = [a.get_text() for a in x.select(".Section-module__container___2mwAW .Table-module__name___1XC4z")]
        pd_month_day = [b.get_text() for b in x.select(".Section-module__container___2mwAW .Table-module__month___2Efxb") * len(pd_news_day)]
        pd_date_day = [c.get_text() for c in x.select(".Section-module__container___2mwAW .Table-module__dayNumber___1DiwN") * len(pd_news_day)]
        pd_wkday_day = [d.get_text() for d in x.select(".Section-module__container___2mwAW .Table-module__weekday___1xUx_") * len(pd_news_day)]
        pd_time_day = [e.get_text() for e in x.select(".Section-module__container___2mwAW .Table-module__time___iPjt3")]
        pd_curr_day = [f.get_text() for f in x.select(".Section-module__container___2mwAW .Currency-module__code___1A32e")]
        pd_impact_day = [g.get_text() for g in x.select(".Section-module__container___2mwAW .Impact-module__pill___2pcuA")]
        pd_forecast_day = [h.get_text() for h in x.select(".Table-module__eventRow___2Z1BD .Table-module__forecast___3F_kh")]
        pd_previous_day = [i.get_text() for i in x.select(".Table-module__eventRow___2Z1BD .Table-module__previous___1PJZ9")]

    #Build list elements into list of dataframes, concatenate dataframes and export to csv 
        df_ffxevents_day = pd.DataFrame.from_dict({
        "pd_year": bpip_yr_dfcastiso,
        "pd_month": pd_month_day,
        "pd_date": pd_date_day,
        "pd_weekday": pd_wkday_day,
        "pd_time": pd_time_day,
        "pd_currency": pd_curr_day,
        "pd_news": pd_news_day,
        "pd_impact": pd_impact_day,
        "pd_forecast": pd_forecast_day,
        "pd_previous": pd_previous_day})
        df_ffxevents_list.append(df_ffxevents_day)
    count = count + 1
df_ffxevents_cat = pd.concat(df_ffxevents_list)
df_ffxevents_cat.to_csv('fxevents_forecast.csv')
count=0
df_ffxevents_cat.head(20)


Forecast Date: 2021-01-03
Forecast Week String: 53
Current Year String: 2021
Current Year: 2020
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2020-W53
Forecast Date: 2021-01-10
Forecast Week String: 01
Current Year String: 2021
Current Year: 2021
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2021-W01
Forecast Date: 2021-01-17
Forecast Week String: 02
Current Year String: 2021
Current Year: 2021
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2021-W02
Forecast Date: 2021-01-24
Forecast Week String: 03
Current Year String: 2021
Current Year: 2021
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2021-W03
Forecast Date: 2021-01-31
Forecast Week String: 04
Current Year String: 2021
Current Year: 2021
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2021-W04
Forecast Date: 2021-02-07
Forecast Week String: 05
Current Year String: 2021
Current Year: 2021
http

,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,2020,Dec,27,Sunday,17:50,JPY,BoJ Summary of Opinions,med,,
1,2020,Dec,27,Sunday,17:50,JPY,Industrial Production MoM Prel,med,2.2%,4.0%
2,2020,Dec,27,Sunday,17:50,JPY,Industrial Production YoY Prel,low,-0.7%,-3.0%
0,2020,Dec,28,Monday,All Day,AUD,Boxing Day (substitute day),low,,
1,2020,Dec,28,Monday,All Day,CAD,Boxing Day (substitute day),low,,
2,2020,Dec,28,Monday,All Day,NZD,Boxing Day (substitute day),low,,
3,2020,Dec,28,Monday,All Day,GBP,Boxing Day (substitute day),low,,
4,2020,Dec,28,Monday,05:00,EUR,French Jobseekers Total,low,"3,570.7k","3,549.7k"
5,2020,Dec,28,Monday,05:00,EUR,French Unemployment Benefit Claims,med,21.0k,-56.6k
6,2020,Dec,28,Monday,08:00,EUR,French 6-Month BTF Auction,low,,


In [115]:
file_path = Path("fxevents_forecast.csv")
forecast_data = pd.read_csv(file_path)
forecast_data.head(20)

,Unnamed: 0,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,0,2020,Dec,27,Sunday,17:50,JPY,BoJ Summary of Opinions,med,NaN,NaN
1,1,2020,Dec,27,Sunday,17:50,JPY,Industrial Production MoM Prel,med,2.2%,4.0%
2,2,2020,Dec,27,Sunday,17:50,JPY,Industrial Production YoY Prel,low,-0.7%,-3.0%
3,0,2020,Dec,28,Monday,All Day,AUD,Boxing Day (substitute day),low,NaN,NaN
4,1,2020,Dec,28,Monday,All Day,CAD,Boxing Day (substitute day),low,NaN,NaN
5,2,2020,Dec,28,Monday,All Day,NZD,Boxing Day (substitute day),low,NaN,NaN
6,3,2020,Dec,28,Monday,All Day,GBP,Boxing Day (substitute day),low,NaN,NaN
7,4,2020,Dec,28,Monday,05:00,EUR,French Jobseekers Total,low,"3,570.7k","3,549.7k"
8,5,2020,Dec,28,Monday,05:00,EUR,French Unemployment Benefit Claims,med,21.0k,-56.6k
9,6,2020,Dec,28,Monday,08:00,EUR,French 6-Month BTF Auction,low,NaN,NaN


In [116]:
forecast_data['Date'] = pd.to_datetime(forecast_data[['pd_month', 'pd_date', 'pd_year']].astype(str).agg('-'.join, axis=1))
forecast_data.set_index(forecast_data['Date'], inplace=True)
forecast_data = forecast_data.drop(columns=["pd_month", "pd_date", "pd_year", "pd_weekday", "Date"])
forecast_data = forecast_data.loc[:, ~forecast_data.columns.str.contains('^Unnamed')]
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2020-12-27,17:50,JPY,BoJ Summary of Opinions,med,NaN,NaN
2020-12-27,17:50,JPY,Industrial Production MoM Prel,med,2.2%,4.0%
2020-12-27,17:50,JPY,Industrial Production YoY Prel,low,-0.7%,-3.0%
2020-12-28,All Day,AUD,Boxing Day (substitute day),low,NaN,NaN
2020-12-28,All Day,CAD,Boxing Day (substitute day),low,NaN,NaN
2020-12-28,All Day,NZD,Boxing Day (substitute day),low,NaN,NaN
2020-12-28,All Day,GBP,Boxing Day (substitute day),low,NaN,NaN
2020-12-28,05:00,EUR,French Jobseekers Total,low,"3,570.7k","3,549.7k"
2020-12-28,05:00,EUR,French Unemployment Benefit Claims,med,21.0k,-56.6k


In [117]:
forecast_data = forecast_data[forecast_data['pd_currency'].isin(['USD', 'EUR', 'GBP', 'JPY'])]
forecast_data = forecast_data[forecast_data['pd_impact'].isin(['high'])]
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,170.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-67.6B,-63.1B
2021-01-08,01:00,EUR,German Balance of Trade,high,17.6B,19.4B
2021-01-08,04:00,EUR,Business Climate,high,NaN,NaN
2021-01-08,07:30,USD,Nonfarm Payrolls,high,112.0k,245.0k
2021-01-08,07:30,USD,Unemployment Rate,high,6.7%,6.7%
2021-01-08,07:30,USD,Average Hourly Earnings MoM,high,0.1%,0.3%
2021-01-13,07:30,USD,Inflation Rate YoY,high,1.2%,1.2%


In [118]:
forecast_data.isnull().mean() * 100

pd_time         0.000000
pd_currency     0.000000
pd_news         0.000000
pd_impact       0.000000
pd_forecast    59.154930
pd_previous    64.788732
dtype: float64

In [119]:
forecast_data["pd_forecast"] = forecast_data["pd_forecast"].fillna(0)
forecast_data["pd_previous"] = forecast_data["pd_previous"].fillna(0)
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,170.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-67.6B,-63.1B
2021-01-08,01:00,EUR,German Balance of Trade,high,17.6B,19.4B
2021-01-08,04:00,EUR,Business Climate,high,0,0
2021-01-08,07:30,USD,Nonfarm Payrolls,high,112.0k,245.0k
2021-01-08,07:30,USD,Unemployment Rate,high,6.7%,6.7%
2021-01-08,07:30,USD,Average Hourly Earnings MoM,high,0.1%,0.3%
2021-01-13,07:30,USD,Inflation Rate YoY,high,1.2%,1.2%


In [120]:
forecast_data.isnull().mean() * 100

pd_time        0.0
pd_currency    0.0
pd_news        0.0
pd_impact      0.0
pd_forecast    0.0
pd_previous    0.0
dtype: float64

In [121]:
forecast_data = forecast_data.rename(columns={
    "pd_time": "Time",
    "pd_currency": "Currency",
    "pd_news": "News", 
    "pd_impact": "Impact",
    "pd_forecast": "Forecast",
    "pd_previous": "Previous"})
forecast_data.to_csv("Data/fxevents_forecast_GSd.csv")
forecast_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous
Date,,,,,,
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,170.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-67.6B,-63.1B
2021-01-08,01:00,EUR,German Balance of Trade,high,17.6B,19.4B
2021-01-08,04:00,EUR,Business Climate,high,0,0
2021-01-08,07:30,USD,Nonfarm Payrolls,high,112.0k,245.0k
2021-01-08,07:30,USD,Unemployment Rate,high,6.7%,6.7%
2021-01-08,07:30,USD,Average Hourly Earnings MoM,high,0.1%,0.3%
2021-01-13,07:30,USD,Inflation Rate YoY,high,1.2%,1.2%


In [122]:
forecast_path = Path("Data/fxevents_forecast_master.csv")
forecastmstr_data = pd.read_csv(forecast_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
forecastmstr_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous,Direction
Date,,,,,,,
2021-01-05,23:00,JPY,Consumer Confidence,high,35,33.7,1
2021-01-06,7:15,USD,ADP National Employment Report,high,170.0k,307.0k,1
2021-01-07,7:30,USD,Balance of Trade,high,-67.6B,-63.1B,1
2021-01-08,1:00,EUR,German Balance of Trade,high,17.6B,19.4B,1
2021-01-08,4:00,EUR,Business Climate,high,0,0,0
2021-01-08,7:30,USD,Nonfarm Payrolls,high,112.0k,245.0k,1
2021-01-08,7:30,USD,Unemployment Rate,high,6.70%,6.70%,-1
2021-01-08,7:30,USD,Average Hourly Earnings MoM,high,0.10%,0.30%,1
2021-01-13,7:30,USD,Inflation Rate YoY,high,1.20%,1.20%,-1


In [123]:
forecastmstr_data["Previous"] = forecastmstr_data["Previous"].str.replace("*", "").str.replace("%", "").str.replace("k", "").str.replace("B", "").str.replace(",", "")
forecastmstr_data["Forecast"] = forecastmstr_data["Forecast"].str.replace("*", "").str.replace("%", "").str.replace("k", "").str.replace("B", "").str.replace(",", "")    
forecastmstr_data['Previous'] = forecastmstr_data['Previous'].astype('float')
forecastmstr_data['Forecast'] = forecastmstr_data['Forecast'].astype('float')
forecastmstr_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous,Direction
Date,,,,,,,
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7,1
2021-01-06,7:15,USD,ADP National Employment Report,high,170.0,307.0,1
2021-01-07,7:30,USD,Balance of Trade,high,-67.6,-63.1,1
2021-01-08,1:00,EUR,German Balance of Trade,high,17.6,19.4,1
2021-01-08,4:00,EUR,Business Climate,high,0.0,0.0,0
2021-01-08,7:30,USD,Nonfarm Payrolls,high,112.0,245.0,1
2021-01-08,7:30,USD,Unemployment Rate,high,6.7,6.7,-1
2021-01-08,7:30,USD,Average Hourly Earnings MoM,high,0.1,0.3,1
2021-01-13,7:30,USD,Inflation Rate YoY,high,1.2,1.2,-1


In [124]:
def pair_menu():
    print("[1] USD/JPY")
    print("[2] EUR/USD")
    print("[3] GBP/USD")
    print("[4] EXIT")

def get_pair_con(target_pair, base, quote):
    print(f"Retrieving 4-week forecast of high impact announcements for {target_pair}...\n")
    # Get 4-week forecast for currency pair
    bpip_pair_cast = bpip_date + bpip_pair_delta
    forecastmstr_filter = forecastmstr_data.loc[bpip_date:bpip_pair_cast]
    forecastmstr_filter_pair = forecastmstr_filter[forecastmstr_filter['Currency'].isin([base, quote])]
    print(forecastmstr_filter_pair)
    
    # Get consensus for base currency - Positive Key
    forecastmstr_filter_base = forecastmstr_filter[forecastmstr_filter['Currency'].isin([base])]
    pos_base_filter = forecastmstr_filter_base.loc[forecastmstr_filter_base['Direction'] == 1]
    base_pos_forcnt1 = sum(pos_base_filter.Forecast > pos_base_filter.Previous) #(default) sum of comparison to determining if news is positive for base currency
    base_neg_forcnt1 = sum(pos_base_filter.Forecast < pos_base_filter.Previous) #sum of comparison to determining if news is negative for base currency
    
    # Get consensus for quoted currency - Positive Key
    forecastmstr_filter_quote = forecastmstr_filter[forecastmstr_filter['Currency'].isin([quote])]
    pos_quote_filter = forecastmstr_filter_quote.loc[forecastmstr_filter_quote['Direction'] == 1]
    quote_pos_forcnt1 = sum(pos_quote_filter.Forecast > pos_quote_filter.Previous) #(default) sum of comparison to determining if news is positive for quote currency
    quote_neg_forcnt1 = sum(pos_quote_filter.Forecast < pos_quote_filter.Previous) #sum of comparison to determining if news is negative for quote currency
   
    # -----------------------------------------------------------------------

    # Get consensus for base currency - Negative Key
    forecastmstr_filter_base = forecastmstr_filter[forecastmstr_filter['Currency'].isin([base])]
    neg_base_filter = forecastmstr_filter_base.loc[forecastmstr_filter_base['Direction'] == -1]
    base_pos_forcnt = sum(neg_base_filter.Forecast > neg_base_filter.Previous) #(default) sum of comparison to determining if news is positive for base currency
    base_neg_forcnt = sum(neg_base_filter.Forecast < neg_base_filter.Previous) #sum of comparison to determining if news is negative for base currency
    
    # Get consensus for quoted currency - Negative Key
    forecastmstr_filter_quote = forecastmstr_filter[forecastmstr_filter['Currency'].isin([quote])]
    neg_quote_filter = forecastmstr_filter_quote.loc[forecastmstr_filter_quote['Direction'] == -1]
    quote_pos_forcnt = sum(neg_quote_filter.Forecast > neg_quote_filter.Previous) #(default) sum of comparison to determining if news is positive for quote currency
    quote_neg_forcnt = sum(neg_quote_filter.Forecast < neg_quote_filter.Previous) #sum of comparison to determining if news is negative for quote currency

    #Total the number of positive vs. negative consensus for the base and quoted currency 
    tot_base_pos_sum = base_pos_forcnt1+base_pos_forcnt
    tot_base_neg_sum = base_neg_forcnt1+base_neg_forcnt
    tot_quote_pos_sum = quote_pos_forcnt1+quote_pos_forcnt
    tot_quote_neg_sum = quote_neg_forcnt1+quote_neg_forcnt
    
    #Compare the number of positive vs. negative consensus for the base and quoted currency and give recommendation
    if (tot_base_neg_sum > tot_base_pos_sum) and (tot_quote_pos_sum > tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall negative for the {base} and positive for the {quote} therefore the recommendation is to sell the {base}/{quote}")
    elif (tot_base_neg_sum < tot_base_pos_sum) and (tot_quote_pos_sum < tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall positive for the {base} and negative for the {quote} therefore the recommendation is to buy {base}/{quote}")
    elif (tot_base_neg_sum > tot_base_pos_sum) and (tot_quote_pos_sum < tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall neutral for the {base}/{quote}. Trade at your own discretion")
    elif (tot_base_neg_sum < tot_base_pos_sum) and (tot_quote_pos_sum > tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall neutral for the {base}/{quote}. Trade at your own discretion")
    elif (tot_base_neg_sum == tot_base_pos_sum) and (tot_quote_pos_sum < tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall positive for the {base} and negative for the {quote} therefore the recommendation is to buy {base}/{quote}")
    elif (tot_base_neg_sum < tot_base_pos_sum) and (tot_quote_pos_sum == tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall positive for the {base} and negative for the {quote} therefore the recommendation is to buy {base}/{quote}")
    elif (tot_base_neg_sum == tot_base_pos_sum) and (tot_quote_pos_sum > tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall negative for the {base} and positive for the {quote} therefore the recommendation is to sell the {base}/{quote}")
    elif (tot_base_neg_sum > tot_base_pos_sum) and (tot_quote_pos_sum == tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall negative for the {base} and positive for the {quote} therefore the recommendation is to sell the {base}/{quote}")
    elif (tot_base_neg_sum == tot_base_pos_sum) and (tot_quote_pos_sum == tot_quote_neg_sum):
        print(f"The consensus for the next 4 weeks is overall neutral for the {base}/{quote}. Trade at your own discretion")

pair_menu()
pair_option = int(input("Select the currency pair you wish to trade: "))

while pair_option != 4:
    if pair_option == 1:
        #get USDJPY news events & provide recommendation
        get_pair_con('USD/JPY', 'USD', 'JPY')
    elif pair_option == 2:
        #get EURUSD news events & provide recommendation
        get_pair_con('EUR/USD', 'EUR', 'USD')
    elif pair_option == 3:
        #get GBPUSD news events & provide recommendation
        get_pair_con('GBP/USD', 'GBP', 'USD')
    else:
        print(f"Invalid selection. please choose a valid option ")

    print()
    pair_menu()
    pair_option = int(input("Select the currency pair you wish to trade: "))

print(f"Thanks for using the FOREX Robo Advisor. Happy Trading!!")

[1] USD/JPY
[2] EUR/USD
[3] GBP/USD
[4] EXIT


Select the currency pair you wish to trade:  1


Retrieving 4-week forecast of high impact announcements for USD/JPY...

             Time Currency                            News Impact  Forecast  \
Date                                                                          
2021-01-05  23:00      JPY             Consumer Confidence   high      35.0   
2021-01-06   7:15      USD  ADP National Employment Report   high     170.0   
2021-01-07   7:30      USD                Balance of Trade   high     -67.6   
2021-01-08   7:30      USD                Nonfarm Payrolls   high     112.0   
2021-01-08   7:30      USD               Unemployment Rate   high       6.7   
2021-01-08   7:30      USD     Average Hourly Earnings MoM   high       0.1   
2021-01-13   7:30      USD              Inflation Rate YoY   high       1.2   
2021-01-13   7:30      USD         Core Inflation Rate YoY   high       1.6   
2021-01-15   7:30      USD                         PPI MoM   high       0.2   
2021-01-15   7:30      USD                Retail Sales MoM 

Select the currency pair you wish to trade:  2


Retrieving 4-week forecast of high impact announcements for EUR/USD...

            Time Currency                            News Impact  Forecast  \
Date                                                                         
2021-01-06  7:15      USD  ADP National Employment Report   high     170.0   
2021-01-07  7:30      USD                Balance of Trade   high     -67.6   
2021-01-08  1:00      EUR         German Balance of Trade   high      17.6   
2021-01-08  4:00      EUR                Business Climate   high       0.0   
2021-01-08  7:30      USD                Nonfarm Payrolls   high     112.0   
2021-01-08  7:30      USD               Unemployment Rate   high       6.7   
2021-01-08  7:30      USD     Average Hourly Earnings MoM   high       0.1   
2021-01-13  7:30      USD              Inflation Rate YoY   high       1.2   
2021-01-13  7:30      USD         Core Inflation Rate YoY   high       1.6   
2021-01-14  3:00      EUR        German Government Budget   high      

Select the currency pair you wish to trade:  3


Retrieving 4-week forecast of high impact announcements for GBP/USD...

             Time Currency                            News Impact  Forecast  \
Date                                                                          
2021-01-06   7:15      USD  ADP National Employment Report   high     170.0   
2021-01-07   7:30      USD                Balance of Trade   high     -67.6   
2021-01-08   7:30      USD                Nonfarm Payrolls   high     112.0   
2021-01-08   7:30      USD               Unemployment Rate   high       6.7   
2021-01-08   7:30      USD     Average Hourly Earnings MoM   high       0.1   
2021-01-13   7:30      USD              Inflation Rate YoY   high       1.2   
2021-01-13   7:30      USD         Core Inflation Rate YoY   high       1.6   
2021-01-15   1:00      GBP    Manufacturing Production MoM   high       0.2   
2021-01-15   1:00      GBP    Manufacturing Production YoY   high      -6.0   
2021-01-15   1:00      GBP                Balance of Trade 

Select the currency pair you wish to trade:  4


Thanks for using the FOREX Robo Advisor. Happy Trading!!


In [ ]:
# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Set start and end dates of one year back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp("2017-01-01", tz="America/Chicago").isoformat()
end_date = pd.Timestamp("2020-12-31", tz="America/Chicago").isoformat()

In [ ]:
# Get 3 years' worth of historical data for target pair

# Get current closing prices for pairs
df_port = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Display sample data
df_port.head(10)

In [ ]:
# Configure a Monte Carlo simulation to forecast 30 years cumulative returns
MC_weighty_dist = MCSimulation(
    portfolio_data = df_port,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
)

# Print the simulation input data
MC_weighty_dist.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_weighty_dist.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
port_line_plot = MC_weighty_dist.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
port_dist_plot = MC_weighty_dist.plot_distribution()